In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT name,
    type
FROM sqlite_master
WHERE type in ('table', 'view');

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT *
FROM invoice
LIMIT 5;

 * sqlite:///chinook.db
Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


In [4]:
%%sql
SELECT DISTINCT(country)
FROM customer;

 * sqlite:///chinook.db
Done.


country
Brazil
Germany
Canada
Norway
Czech Republic
Austria
Belgium
Denmark
USA
Portugal


In [5]:
%%sql
SELECT g.name,
       COUNT(t.track_id) AS numb_tracks
 FROM track AS t
      INNER JOIN genre AS g
      ON t.genre_id = g.genre_id
GROUP BY 1
ORDER BY 2 DESC;
    

 * sqlite:///chinook.db
Done.


name,numb_tracks
Rock,1297
Latin,579
Metal,374
Alternative & Punk,332
Jazz,130
TV Shows,93
Blues,81
Classical,74
Drama,64
R&B/Soul,61


In [6]:
%%sql
WITH usa AS
    (
    SELECT t.*
    FROM track AS t
    INNER JOIN invoice_line AS il
    ON t.track_id = il.track_id
    INNER JOIN invoice AS i
    ON il.invoice_id = i.invoice_id
    INNER JOIN customer AS c
    ON i.customer_id = c.customer_id
    WHERE c.country = 'USA'
    )
SELECT g.name,
       COUNT(usa.track_id) AS numb_tracks,
       (CAST(COUNT(usa.track_id) AS float)/(SELECT COUNT(*) FROM usa)) 
                                            AS percentage
    FROM usa
    INNER JOIN genre AS g
    ON usa.genre_id = g.genre_id
    GROUP BY 1
    ORDER BY 2 DESC;
    

 * sqlite:///chinook.db
Done.


name,numb_tracks,percentage
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


No hip-hop, because it has the lowest share at usa market

In [7]:
%%sql
SELECT e.first_name || ' ' ||e.last_name AS employee_name,
       SUM(i.total) AS total_sales, 
       COUNT(i.customer_id) AS total_customers
FROM employee AS e
INNER JOIN customer AS c
ON e.employee_id = c.support_rep_id
INNER JOIN invoice AS i
ON c.customer_id = i.customer_id
GROUP BY 1
ORDER BY 2 DESC;    

 * sqlite:///chinook.db
Done.


employee_name,total_sales,total_customers
Jane Peacock,1731.5100000000039,212
Margaret Park,1584.0000000000034,214
Steve Johnson,1393.920000000002,188


The difference in total sales can be described with the different number of customers.

In [8]:
%%sql
SELECT new_country AS country,
       SUM(tot_customers) AS tot_customers,
       ROUND(SUM(tot_sales), 2) AS tot_sales,
       ROUND((SUM(tot_sales) / SUM(tot_customers)), 2) AS sales_per_customer,
       ROUND(AVG(avg_order), 2) AS avg_order
FROM 
(
WITH by_country AS
(
SELECT CASE
            WHEN COUNT(DISTINCT(c.customer_id)) = 1 THEN 'Other'
            ELSE c.country
            END AS new_country,
       COUNT(DISTINCT(c.customer_id)) AS tot_customers,
       SUM(i.total) AS tot_sales,
       CAST(SUM(i.total) AS float)/COUNT(i.invoice_id) AS avg_order
FROM customer AS c
INNER JOIN invoice i
ON c.customer_id = i.customer_id
GROUP BY c.country
)
SELECT *,
       CASE 
            WHEN new_country = 'Other' THEN 1
            ELSE 0
            END AS sort
FROM by_country
)
GROUP BY new_country
ORDER BY sort ASC, tot_sales DESC;

        
       

 * sqlite:///chinook.db
Done.


country,tot_customers,tot_sales,sales_per_customer,avg_order
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.57,8.72
Other,15,1094.94,73.0,7.45


We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [9]:
%%sql
SELECT a.album_id,
       COUNT(t.track_id) AS tracks_per_album 
  FROM album AS a
INNER JOIN track AS t
ON a.album_id = t.album_id
 GROUP BY a.album_id
 LIMIT 10;

 * sqlite:///chinook.db
Done.


album_id,tracks_per_album
1,10
2,1
3,3
4,8
5,15
6,13
7,12
8,14
9,8
10,14


In [10]:
%%sql
    SELECT  i.invoice_id,
            COUNT(t.track_id) AS tracks_per_invoice_per_album,
            t.album_id
       FROM invoice AS i
       INNER JOIN invoice_line AS il
       ON i.invoice_id = il.invoice_id
       INNER JOIN track AS t
       ON il.track_id = t.track_id
    GROUP BY t.album_id, i.invoice_id

 * sqlite:///chinook.db
Done.


invoice_id,tracks_per_invoice_per_album,album_id
21,1,1
23,10,1
35,1,1
42,1,1
47,1,1
63,2,1
67,1,1
107,1,1
129,1,1
203,1,1


In [21]:
%%sql
SELECT i.invoice_id,
               COUNT(DISTINCT(t.album_id)) AS numb_alb_per_invoice
        FROM track AS t
        INNER JOIN invoice_line AS il
        ON t.track_id = il.track_id
        INNER JOIN invoice i
        ON il.invoice_id = i.invoice_id
        GROUP BY i.invoice_id

 * sqlite:///chinook.db
Done.


invoice_id,numb_alb_per_invoice
1,1
2,10
3,2
4,8
5,1
6,2
7,11
8,10
9,9
10,2


In [25]:
%%sql
SELECT COUNT(a_p.album_purchase_id) AS album_purchase_numb,
       CAST(COUNT(a_p.album_purchase_id) AS float)/ COUNT(DISTINCT(i.invoice_id)) AS album_purchase_perc
FROM invoice i
LEFT JOIN 
(
    WITH tracks_per_invoice AS
    (
     SELECT  i.invoice_id,
            COUNT(t.track_id) AS tracks_per_invoice,
            t.album_id
       FROM invoice AS i
       INNER JOIN invoice_line AS il
       ON i.invoice_id = il.invoice_id
       INNER JOIN track AS t
       ON il.track_id = t.track_id
    GROUP BY t.album_id, i.invoice_id),
    
    tracks_per_album AS
    (
    SELECT a.album_id,
           COUNT(t.track_id) AS tracks_per_album 
      FROM album AS a
    INNER JOIN track AS t
    ON a.album_id = t.album_id
     GROUP BY a.album_id
    ),
    albums_per_invoice AS
    (
        SELECT i.invoice_id,
               COUNT(DISTINCT(t.album_id)) AS numb_alb_per_invoice
        FROM track AS t
        INNER JOIN invoice_line AS il
        ON t.track_id = il.track_id
        INNER JOIN invoice i
        ON il.invoice_id = i.invoice_id
        GROUP BY i.invoice_id
    )
SELECT t_i.invoice_id AS album_purchase_id   
FROM tracks_per_invoice AS t_i
INNER JOIN tracks_per_album AS t_a
ON t_i.album_id = t_a.album_id
INNER JOIN albums_per_invoice AS a_i
ON t_i.invoice_id = a_i.invoice_id
WHERE (INSTR(" "|| t_a.album_id || t_a.tracks_per_album || " ", 
            " " || t_i.album_id || t_i.tracks_per_invoice || " ")!=0) AND (a_i.numb_alb_per_invoice = 1)
ORDER BY t_i.invoice_id
) AS a_p
ON i.invoice_id = a_p.album_purchase_id;

 * sqlite:///chinook.db
Done.


album_purchase_numb,album_purchase_perc
114,0.18566775244299674


Which artist is used in the most playlists?
TOP-5

In [27]:
%%sql
SELECT ar.artist_id,
       ar.name,
       COUNT(playlist_id) AS number_of_playlists
FROM artist AS ar
INNER JOIN album AS al 
ON ar.artist_id = al.artist_id
INNER JOIN track AS t
ON al.album_id = t.album_id
INNER JOIN playlist_track AS p_t
ON t.track_id = p_t.track_id
GROUP BY ar.artist_id
ORDER BY number_of_playlists DESC
LIMIT 5;

 * sqlite:///chinook.db
Done.


artist_id,name,number_of_playlists
90,Iron Maiden,516
150,U2,333
50,Metallica,296
22,Led Zeppelin,252
58,Deep Purple,226


How many tracks have been purchased vs not purchased?

In [34]:
%%sql
WITH p_t AS
    (
     SELECT COUNT(DISTINCT(t.track_id)) AS purchased_tracks
    FROM track AS t
    INNER JOIN invoice_line AS il
    ON t.track_id = il.track_id
    )
SELECT  p_t.purchased_tracks,
        COUNT(DISTINCT(t.track_id)) - p_t.purchased_tracks AS not_purchased,
        COUNT(DISTINCT(t.track_id)) AS total
FROM track AS t, p_t;

 * sqlite:///chinook.db
Done.


purchased_tracks,not_purchased,total
1806,1697,3503


Is the range of tracks in the store reflective of their sales popularity?


In [48]:
%%sql
WITH sold_genre AS
    (
    SELECT t.genre_id,
           COUNT(DISTINCT(t.track_id)) AS sold_track_by_genre
    FROM track AS t
    INNER JOIN invoice_line AS il
    ON t.track_id = il.track_id
    GROUP BY t.genre_id
    ORDER BY sold_track_by_genre
    )
SELECT  g.name,
        COUNT(DISTINCT(t.track_id)) AS total_tracks_by_genre,
        s_g.sold_track_by_genre,
        ROUND(CAST(s_g.sold_track_by_genre AS float)/COUNT(DISTINCT(t.track_id)), 2) AS sold_from_total
FROM track AS t
INNER JOIN sold_genre AS s_g
ON t.genre_id = s_g.genre_id
INNER JOIN genre AS g
ON t.genre_id=g.genre_id
GROUP BY g.name
ORDER BY 4 DESC;

 * sqlite:///chinook.db
Done.


name,total_tracks_by_genre,sold_track_by_genre,sold_from_total
Easy Listening,24,24,1.0
Electronica/Dance,30,29,0.97
R&B/Soul,61,55,0.9
Alternative,40,34,0.85
Rock,1297,915,0.71
Blues,81,56,0.69
Metal,374,238,0.64
Hip Hop/Rap,35,21,0.6
Alternative & Punk,332,176,0.53
Pop,48,25,0.52


Conparison on how much tracks from each genre are at the store vs were purchased showed that there are some genre that are over-represented in regard of the demand: that are drama and TV shows genres having only 2 % sold while there are top-3 genres with 90%: Easy Listening, Electronica/Dance and R&B/Soul. 

Do protected vs non-protected media types have an effect on popularity?

In [61]:
%%sql

SELECT m_t.protected,
        COUNT(t.track_id) AS number
FROM track AS t
INNER JOIN (SELECT *,CASE
                        WHEN media_type.name LIKE '%Protected%' OR '%Purchased%' THEN 'yes'
                        ELSE 'no'
                        END AS protected
                        FROM media_type) AS m_t
ON t.media_type_id = m_t.media_type_id
LEFT JOIN invoice_line AS il
ON t.track_id = il.track_id
WHERE il.invoice_line_id IS NULL
GROUP BY m_t.protected
    

 * sqlite:///chinook.db
Done.


protected,number
no,1400
yes,297


In [62]:
%%sql

SELECT m_t.protected,
        COUNT(t.track_id) AS number
FROM track AS t
INNER JOIN (SELECT *,CASE
                        WHEN media_type.name LIKE '%Protected%' OR '%Purchased%' THEN 'yes'
                        ELSE 'no'
                        END AS protected
                        FROM media_type) AS m_t
ON t.media_type_id = m_t.media_type_id
LEFT JOIN invoice_line AS il
ON t.track_id = il.track_id
WHERE il.invoice_line_id NOT NULL
GROUP BY m_t.protected

 * sqlite:///chinook.db
Done.


protected,number
no,4315
yes,442
